# **CNN + 전처리**

# 라이브러리 및 데이터 불러오기

In [ ]:
#import Libraries -1
import pandas as pd
import numpy as np
import re

import nltk
import nltk.data
from nltk import word_tokenize
from nltk.corpus import stopwords

from sklearn import metrics, preprocessing, pipeline, model_selection, naive_bayes
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier, LogisticRegression
import xgboost as xgb

import time

from keras import backend as K
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import GlobalAveragePooling1D, Conv1D, MaxPooling1D, Flatten
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import sequence, text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping

In [ ]:
#importing libraries-2
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings(action='ignore')

In [ ]:
#데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test_x.csv')
sample = pd.read_csv('/content/drive/MyDrive/sample_submission.csv')

In [ ]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [ ]:
#작가별 문장 개수 파악하기
train['author'].value_counts()

3    15063
0    13235
2    11554
4     7805
1     7222
Name: author, dtype: int64

In [ ]:
test.shape

(19617, 2)

# Feature Engineering 데이터 전처리

In [ ]:
# Importing the libraries
import numpy as np
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [ ]:
import nltk
stopwords = nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# replace
train['text'] = train['text'].str.replace('[^a-zA-Z0-9]', ' ')
test['text'] =test['text'].str.replace('[^a-zA-Z0-9]', ' ')

## text 내 단어 개수
train["num_words"] = train["text"].apply(lambda x: len(str(x).split()))
test["num_words"] = test["text"].apply(lambda x: len(str(x).split()))

## text 내 고유단어 개수
train["num_unique_words"] = train["text"].apply(lambda x: len(set(str(x).split())))
test["num_unique_words"] = test["text"].apply(lambda x: len(set(str(x).split())))

## text 내 character 개수
train["num_chars"] = train["text"].apply(lambda x: len(str(x)))
test["num_chars"] = test["text"].apply(lambda x: len(str(x)))

## text 내 stopword 개수
eng_stopwords = nltk.corpus.stopwords.words('english')
train["num_stopwords"] = train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_stopwords"] = test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))


In [ ]:
## text 내 punctuation 개수
import string
train["num_punctuations"] =train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_punctuations"] =test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## text 내 upper case 단어 개수
train["num_words_upper"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test["num_words_upper"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## text 내 title case 단어 개수
train["num_words_title"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test["num_words_title"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## text 내 평균 단어 길이
train["mean_word_len"] = train["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test["mean_word_len"] = test["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [ ]:
train??

,index,text,author,num_punctuations,num_words_upper,num_words_title,mean_word_len
0,0,"He was almost choking. There was so much, so m...",3,8,0,4,4.239130
1,1,"“Your sister asked for it, I suppose?”",2,2,1,2,4.571429
2,2,"She was engaged one day as she walked, in per...",1,9,0,4,4.614035
3,3,"The captain was in the porch, keeping himself ...",4,18,0,7,4.517241
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3,13,0,4,4.871795
...,...,...,...,...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2,5,1,4,4.666667
54875,54875,"I told my plan to the captain, and between us ...",4,2,1,1,4.277778
54876,54876,"""Your sincere well-wisher, friend, and sister...",1,7,1,1,6.375000
54877,54877,“Then you wanted me to lend you money?”,3,1,0,1,4.000000


In [ ]:
# Clean text
def clean_text(text):
    return re.sub('[^a-zA-Z]', ' ', text)

In [ ]:
train['text_cleaned'] = train['text'].apply(lambda x: clean_text(x))
test['text_cleaned'] = test['text'].apply(lambda x: clean_text(x))

In [ ]:
# stopword 제거
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

In [ ]:
# text feature 추출
def extract_features(df):
    df['len'] = df['text'].apply(lambda x: len(x))
    df['n_words'] = df['text'].apply(lambda x: len(x.split(' ')))
    df['n_.'] = df['text'].str.count('\.')
    df['n_...'] = df['text'].str.count('\...')
    df['n_,'] = df['text'].str.count('\,')
    df['n_:'] = df['text'].str.count('\:')
    df['n_;'] = df['text'].str.count('\;')
    df['n_-'] = df['text'].str.count('\-')
    df['n_?'] = df['text'].str.count('\?')
    df['n_!'] = df['text'].str.count('\!')
    df['n_\''] = df['text'].str.count('\'')
    df['n_"'] = df['text'].str.count('\"')
    df.drop(['text_cleaned'], axis=1, inplace=True)

In [ ]:
print('Processing train...')
extract_features(train)
print('Processing test...')
extract_features(test)

Processing train...
Processing test...


In [ ]:
## TF-IDF 적용
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), min_df =0,max_df=0.9,lowercase=False, use_idf=True)
train_tfidf = tfidf_vec.fit_transform(train['text'].values.tolist())
test_tfidf = tfidf_vec.fit_transform(test['text'].values.tolist())
print('Tfidf_train:',train_tfidf.shape)
print('Tfidf_test:',test_tfidf.shape)

Tfidf_train: (54879, 2132503)
Tfidf_test: (19617, 1773623)


# Modeling

In [ ]:
# impoting libraries for CNN modeling
from keras.layers import Embedding, GRU, Dense, Flatten, Dropout
from keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.metrics import log_loss
import gc
print('import keras done')

import keras done


In [ ]:
#importing libraries for NB features
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [ ]:
# importing libraries for 1D CNN
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Embedding, Dropout, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model, to_categorical

In [ ]:
# 데이터 불러오기
X_train = train['text'].values
X_test = test['text'].values
y = train['author'].values
print(X_train.shape, X_test.shape, y.shape)

(54879,) (19617,) (54879,)


In [ ]:
X_train[:3]

array(['He was almost choking  There was so much  so much he wanted to say  but strange exclamations were all that came from his lips  The Pole gazed fixedly at him  at the bundle of notes in his hand  looked at odin  and was in evident perplexity ',
       ' Your sister asked for it  I suppose  ',
       ' She was engaged one day as she walked  in perusing Jane s last letter  and dwelling on some passages which proved that Jane had not written in spirits  when  instead of being again surprised by Mr  odin  she saw on looking up that odin was meeting her  Putting away the letter immediately and forcing a smile  she said '],
      dtype=object)

In [ ]:
#parameter setting
vocab_size = 20000
embedding_dim = 64
max_length = 500
padding_type='post'

In [ ]:
# 문장 tokenization
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
# text 내 단어들을 숫자 sequence 형태로 변환
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
trn = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
tst = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)
print(trn.shape, tst.shape)

(54879, 500) (19617, 500)


In [ ]:
# K-Fold 함수 설정
n_fold = 10
n_class = 5
seed = 42
cv = StratifiedKFold(n_splits=n_fold, shuffle=True,random_state = seed)

In [ ]:
#CNN 모델 설계
def get_model():
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=max_length),
        Dropout(.5),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),
        Conv1D(128, 7, padding="valid", activation="relu", strides=3),    
        GlobalMaxPooling1D(),
        Dense(128, activation='relu'),
        Dropout(.5),
        Dense(n_class, activation='softmax')
    ])
    
    # compile model
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(learning_rate=.005))
    return model

In [ ]:
# CNN 모델 적용(fitting model)
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3,
                       verbose=1, mode='min', baseline=None, restore_best_weights=True) #EarlyStopping 설정

    clf = get_model()    
    clf.fit(trn[i_trn], 
            to_categorical(y[i_trn]),
            validation_data=(trn[i_val], to_categorical(y[i_val])),
            epochs=10,
            batch_size=512,
            callbacks=[es]) 
    p_val[i_val, :] = clf.predict(trn[i_val]) #Evaluating model(validation data에 적용)
    p_tst += clf.predict(tst) / n_fold #testing model

training model for CV #1
Epoch 1/10
97/97 [==============================] - 129s 1s/step - loss: 1.4676 - val_loss: 1.0413
Epoch 2/10
97/97 [==============================] - 128s 1s/step - loss: 0.9523 - val_loss: 0.8569
Epoch 3/10
97/97 [==============================] - 126s 1s/step - loss: 0.7258 - val_loss: 0.7808
Epoch 4/10
97/97 [==============================] - 129s 1s/step - loss: 0.5920 - val_loss: 0.7792
Epoch 5/10
97/97 [==============================] - 127s 1s/step - loss: 0.4976 - val_loss: 0.8046
Epoch 6/10
97/97 [==============================] - 126s 1s/step - loss: 0.4481 - val_loss: 0.8508
Epoch 7/10
97/97 [==============================] - 126s 1s/step - loss: 0.3990 - val_loss: 0.8529
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
training model for CV #2
Epoch 1/10
97/97 [==============================] - 124s 1s/step - loss: 1.5087 - val_loss: 1.2606
Epoch 2/10
97/97 [==============================] - 123s 1s/step - loss: 1

In [ ]:
# Calculating Log_Loss
print(f'Log Loss (CV): {log_loss(pd.get_dummies(y), p_val):8.4f}')

Log Loss (CV):   0.7610


In [ ]:
print(clf.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 500, 64)           1280000   
_________________________________________________________________
dropout_18 (Dropout)         (None, 500, 64)           0         
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 165, 128)          57472     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 53, 128)           114816    
_________________________________________________________________
global_max_pooling1d_9 (Glob (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_19 (Dropout)         (None, 128)              